# Multi-agent Collaboration for Financial Analysis 📊 


## Introduction 
In this notebook, you will learn `how to make agents collaborate with each other to perform financial analysis`. By leveraging multiple agents, we are able to make agents collaborate each other and further we can automate and optimize the process of analyzing market data, developing trading strategies, executing trades, and managing risks.


**Note** If you're running this notebook on your own machine, you can install the following:
```Py
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Suppress warnings for a cleaner output
import warnings
warnings.filterwarnings('ignore')

### Import libraries, APIs and LLM

In [2]:
# Importing necessary classes from the crewai library
from crewai import Agent, Task, Crew

**Note** _Agent_ represents an agent with specific roles and goals. _Task_ represents a task that the agent needs to perform. _Crew_ represents a group of agents working together on tasks.    

Load the environment variables from a .env file to access the OpenAI API key. For more info [.env.example](https://github.com/micag2025/DeepLearningAI-CrewAI/blob/main/Multi_AI_Agent_Systems_with_CrewAI/Mutli_Agent_customer_support_automation/customer_support.ipynb)

In [3]:
# Import the required libraries
import os
from utils import get_openai_api_key, get_serper_api_key

# Fetch the OpenAI API key using a utility function and store it in a variable
openai_api_key = get_openai_api_key()

#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
# Set the environment variable for the OpenAI model name to 'gpt-4-turbo'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

# Fetch the Serper API key using a utility function and set it as an environment variable
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
# Import crewAI tools for web scraping and search functionality
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

**Note** _ScrapeWebsiteTool_ is a Tool for scraping data from websites. The _SerperDevTool_ is a  Tool for performing searches using the Serper API

## Creating Agents

- Agent 1 : Data Analyst Agent

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

- Agent 2: Trading Strategy Agent

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

- Agent 3 : Execution Agent

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

- Agent 4 : Risk Management Agent

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

- Task for Data Analyst Agent: Analyze Market Data

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

- Task for Trading Strategy Agent: Develop Trading Strategies

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


- Task for Trade Advisor Agent: Plan Trade Execution

In [ ]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

- Task for Risk Advisor Agent: Assess Trading Risks

In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

- Example : Moderate-Risk Day Trading Strategy with News Sensitivity

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL to identify trends and predict market movements. I should utilize the skills of the Data Analyst and Trading Strategy Developer to gather insights and alerts about significant market opportunities or threats.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for AAPL",
    "context": "The Data Analyst is responsible for collecting and analyzing market data for AAPL using statistical modeling and machine learning techniques. They should focus on identifying trends and predicting market movements to provide insights and alerts about si

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

Introduction:
This report provides a comprehensive risk analysis for the proposed trading strategies and execution plans for AAPL. The analysis includes an evaluation of potential risks associated with the Long-Term Investment Strategy, Swing Trading Strategy, Options Trading Strategy, and Sector Rotation Strategy, along with suggested mitigation strategies to enhance our trading approach.

Key Risks and Mitigation Strategies:

1. Market Volatility:
Risk: Fluctuations in the market environment, influenced by events like the SVB failure, can impact trading strategies, especially Options and Swing Trading.
Mitigation: Implement effective risk management techniques, such as stop-loss orders and diversification, to mitigate the impact of market volatility on trading outcomes.

2. AAPL Performance:
Risk: While AAPL has shown strong performance, there is a risk of market corrections that could affect short-term trading strategies.
Mitigation: Regularly review and adjust trading strategies to align with AAPL's performance trends and market dynamics, ensuring a proactive approach to potential corrections.

3. Long-Term Investment Strategy:
Risk: Potential market corrections and changes in technology sector dynamics could affect the growth prospects of AAPL under this strategy.
Mitigation: Conduct periodic reviews of the Long-Term Investment Strategy, considering market trends and company performance to make informed adjustments and minimize risks.

4. Swing Trading Strategy:
Risk: High market volatility poses a risk to the Swing Trading Strategy, requiring careful risk management and continuous monitoring.
Mitigation: Set clear entry and exit points, use technical analysis tools for decision-making, and implement appropriate risk-reward ratios to optimize trading outcomes and mitigate risks.

5. Options Trading Strategy:
Risk: Volatility in the market presents both risks and opportunities for Options Trading, requiring strategic selection and risk management.
Mitigation: Utilize advanced options strategies, such as spreads and collars, to manage risk exposure and capitalize on price movements while minimizing potential losses.

6. Sector Rotation Strategy:
Risk: Changes in the technology sector's performance due to economic shifts or technological advancements could impact the Sector Rotation Strategy.
Mitigation: Stay informed about sector trends, monitor competitor performance, and be prepared to rotate investments based on market developments to maximize returns and minimize sector-related risks.

Conclusion:
Effective risk management is essential for navigating the potential risks associated with trading strategies for AAPL. By implementing the suggested mitigation strategies and continuously monitoring market dynamics, we can optimize our trading approach and enhance profitability while mitigating risks effectively.

### Extra investigation

To create another reliable scenario in your CrewAI-based multi-agent financial analysis, you can modify the parameters based on your desired trading approach and risk profile. Here’s how you can tweak them:

Key Adjustments for Different Scenarios

- _Stock Selection (stock_selection)_  
Change the stock based on market conditions or industry preferences.
Example: 'TSLA' (Tesla) for a high-volatility scenario or 'JNJ' (Johnson & Johnson) for a more stable investment.
Initial Capital (initial_capital)

- _Adjust the capital to simulate different investor types_.
Example: Lower value (e.g., 5000) for small retail investors or higher value (e.g., 500000) for institutional-style trading.

- _Risk Tolerance (risk_tolerance)_:    
   - 'Low' → More conservative, lower-risk stocks & safe trading strategies.
   - 'Medium' → Balanced approach (mix of growth & stability).  
   - 'High' → Aggressive, higher volatility, higher reward strategies.  

- _Trading Strategy (trading_strategy_preference)_:  
   -'Day Trading' → High-frequency, short-term trades.
   - 'Swing Trading' → Medium-term trades, holding positions for days/weeks
   - 'Long-Term Investing' → Investing in stocks for months/years.

- _News Impact (news_impact_consideration)_:  
   - True → Incorporates news sentiment analysis into decision-making.  
   - False → Ignores external news, relies solely on technical analysis.

- Example: More Conservative Long-Term Scenario

In [17]:
financial_trading_inputs = {
    'stock_selection': 'JNJ',  # Stable stock
    'initial_capital': '200000',  # Higher capital for long-term investment
    'risk_tolerance': 'Low',  # Conservative approach
    'trading_strategy_preference': 'Long-Term Investing',  # Holding stocks for long periods
    'news_impact_consideration': False  # Ignores short-term news fluctuations
}


In [18]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (JNJ). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for JNJ to the Data Analyst, as they are best equipped to handle this type of work.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for JNJ using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for JNJ."}

> Entering new CrewAgentExecutor chain...
Thought: To monitor and analyze market data for JNJ (Johnson & Johnson), I need to gather the latest financial data and news regarding the company. I should start by searching the internet for the most recent financial reports and news articl

In [19]:
# Display the final result as Markdown.
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Johnson & Johnson (JNJ):

Based on the financial results reported for the fourth quarter and full year 2023, Johnson & Johnson (JNJ) has shown significant growth in sales and earnings. The operational sales growth for Innovative Medicine and MedTech segments has been strong, supported by key products and acquisitions. The financial health indicators, including revenue growth, net income, market capitalization, and dividend yield, reflect a solid foundation for long-term investment.

However, potential risks associated with the proposed trading strategies and execution plans for JNJ include market volatility, ongoing litigation, and high-stakes developments. To mitigate these risks, it is recommended to closely monitor market trends, diversify the portfolio, and adjust the allocation of shares during price fluctuations. Regular monitoring of financial results and news updates is essential to make informed decisions.

Overall, JNJ's strategic plan aims to capitalize on operational performance and medical innovations for gradual stock value appreciation. With a moderate risk tolerance, long-term investors can benefit from the company's strong position in the healthcare sector. By implementing proactive risk management strategies and staying informed about market dynamics, investors can navigate potential challenges and capitalize on growth opportunities within the JNJ portfolio.

- Example: High-Risk, High-Reward Day Trading

In [20]:
financial_trading_inputs = {
    'stock_selection': 'TSLA',  # Volatile stock
    'initial_capital': '50000',  # Mid-level capital for active trading
    'risk_tolerance': 'High',  # Aggressive trading style
    'trading_strategy_preference': 'Day Trading',  # Frequent trades
    'news_impact_consideration': True  # Market-sensitive trades
}


In [21]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (TSLA). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for TSLA to identify trends and predict market movements. I should delegate tasks to the appropriate team members and ask any necessary questions to ensure we are on track to provide insights and alerts about significant market opportunities or threats for TSLA.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for TSLA using statistical modeling and machine learning",
    "context": "Our goal is to identify trends and predict market movements for TSLA. Make sure to provide insights and alerts about significant market opportunities or threats

In [22]:
# Display the final result as Markdown.
from IPython.display import Markdown
Markdown(result)

The comprehensive risk analysis report for TSLA's proposed trading strategies and execution plans reveals several key insights:

1. Market Volatility and Stock Performance: TSLA's stock has exhibited significant fluctuations, indicating a volatile environment that could impact trade entry and exit points.

2. Legal and Regulatory Developments: Tesla's plans to launch robotaxis in Texas by June highlight the regulatory environment differences between Texas and California. The hands-off approach in Texas poses legal and safety risks for Tesla, particularly regarding crash liability and unproven driverless technologies.

3. Earnings Reports and Legal Developments: Tesla's upcoming earnings reports and legal developments regarding autonomous technologies raise concerns about meeting ambitious timelines and technology capabilities, potentially leading to legal and safety issues.

4. Risk Management Strategies: The use of stop-loss orders and diversification strategies are crucial to manage potential losses in the volatile market environment.

5. Real-Time Data and News: Continuous monitoring of real-time market data and news is essential to adapt strategies quickly to sudden market changes and external factors that could impact Tesla's stock.

In conclusion, while there are opportunities for profit in trading TSLA, the associated risks due to market volatility, legal and regulatory challenges, as well as the need for agile risk management strategies must be carefully considered to align with the organization's risk tolerance and trading preferences.

**Note** All these examples use a `sequential workflow`. Sequential Execution (Default). Thus, each agent waits for the previous agent to complete its task before proceeding. This ensures step-by-step validation and structured decision-making. However, the sequential workflow, neverless it is good for controlled environments, may be slower in fast-moving markets.

✔ Example Flow:

In [23]:
from IPython.display import display, HTML

# Load external CSS file
css_file_path = "style.css"
display(HTML(f'<link rel="stylesheet" type="text/css" href="{css_file_path}">'))

# HTML Content
flow_html = """
<div class='flow-container'>
    <span class='flow-step'>Data Analyst</span>
    <span class='arrow'>→</span>
    <span class='flow-step'>Strategy Agent</span>
    <span class='arrow'>→</span>
    <span class='flow-step'>Risk Manager</span>
    <span class='arrow'>→</span>
    <span class='flow-step'>Execution Agent</span>
</div>
"""

display(HTML(flow_html))

Since we are using CrewAI for financial trading, it would be more recommend using a `hybrid approach` to speed up data processing & risk assessment (parallel) and keep strategy formulation & execution structured (sequential).

- Hybrid Execution   
Some agents run in parallel, others remain sequential where needed. For instance, we may consider an example in which Data Analyst & Risk Manager work in parallel, Strategy Agent waits for both before deciding and further Execution Agent waits for final approval
Let's see the optimized Flow:

In [24]:
from IPython.display import display, HTML

# Load external CSS file
css_file_path = "style.css"
display(HTML(f'<link rel="stylesheet" type="text/css" href="{css_file_path}">'))

# HTML Content
flow_html = """
<div class='flow-container'>

    <!-- Parallel Execution Row -->
    <div class='parallel-container'>
        <div class='flow-step'>Data Analyst</div>
        <span class='arrow'>⇄</span>
        <div class='flow-step'>Risk Manager</div>
    </div>

    <!-- Sequential Execution -->
    <div class='flow-row'>
        <span class='arrow'>↓</span>
    </div>

    <div class='flow-row'>
        <div class='flow-step'>Strategy Agent</div>
    </div>

    <div class='flow-row'>
        <span class='arrow'>↓</span>
    </div>

    <div class='flow-row'>
        <div class='flow-step'>Execution Agent</div>
    </div>

</div>
"""

display(HTML(flow_html))


This approch is best for Day trading systems with risk-sensitive adjustments, allowing balancing speed with decision accuracy and Adaptive trading bots that need real-time risk monitoring.

## Conclusion
By utilizing multiple agents to collaborate on financial analysis tasks, we can `automate and optimize the process of monitoring market data, developing trading strategies, executing trades, and managing risks`. This approach leverages the strengths of each agent to create a robust and efficient trading system.